In [190]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

# format sig figs
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [191]:
OUTPUT_DIR = r'J:\Projects\FasTrips\obs\output\OBS_fasttrips_demand_v1.1_stochastic_iter1_nocap_30000'

# obs_links_dir = r'R:\FastTrips\FT Repo\All input & output files\OBS_FToutput.csv'
obs_links_dir = r'..\data\obs\obs_links.csv'

# Probability threshold for observed paths (are observed paths assigned above/below this value by fast-trips)
threshold = 0.3

# Compare observed path to pathset based on modes, agency, or route
# comparison_field = 'path_modes'
comparison_field = 'path_agencies'
# comparison_field = 'path_routes'

non_transit_modes = ['transfer','walk_access','walk_egress','bike_access','bike_egress',
                     'PNR_access','PNR_egress','KNR_access','KNR_egress']
transit_mode_list = ['local_bus','commuter_rail','express_bus','ferry','heavy_rail','light_rail','premium_bus']

In [192]:
def load_df(data, unique_fields, record_type=None):
    '''Load text data as df, create unique trip record ID, and tag as model/observed record'''
    df = pd.read_csv(data)
    if record_type != None:
        df['record_type'] = record_type    # tag as model/observed record

    # Convert all specified unique_fields to string and concatenate as new unique_id field 
#     df[unique_fields] = pd.DataFrame([df[col].astype('int').astype('str') for col in unique_fields]).T
#     df['unique_id'] = df[unique_fields].apply(lambda x: '_'.join(x), axis=1)
    df['unique_id'] = df['person_id']
    
    return df

In [193]:
def append(*args):
    '''Union dataframes with similar structures'''
    df = pd.DataFrame()
    for data in args:
        df = df.append(data)

    return df

In [194]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    miles = 3956 * c
    return miles

In [195]:
def select_common_records(df1,df2,field):
    '''Return dataframe of matching, common records only.
       Example, person 1034 exists in df1, but not in df2, so new copy of df1 without 1034 is created
    '''
    df1 = df1[df1[field].isin(df2[field])]
    df2 = df2[df2[field].isin(df1[field])]

    return df1, df2

In [196]:
def add_transit_agency(df, routes):

    df = pd.merge(left=df,right=routes[['route_id','agency_id']],on='route_id',how='left')

    df['agency'] = df['agency_id']
    df.drop('agency_id',axis=1)
    df.fillna("",inplace=True)
    df.reset_index(inplace=True)

    return df

In [197]:
def produce_path_fields(df, group):
    '''
    Concatenate set of fields for pathset_links, e.g. ('bart caltrain') for 2-leg transit trip
    Produce concatenated fields for routes, modes, agencies, all components (stops, modes, & routes)
    '''
    # create "path_routes"

    for field in ['route_id','mode','agency','A_id','B_id']:
        df[field] = df[field].astype('str')
        df[field] = df[field].fillna("")
        df[field] = df[field].replace('nan',"")

    df['path_routes'] = df['route_id'].apply(lambda x: x.strip())
    path_routes = pd.DataFrame(df.groupby(group)['path_routes'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    result_df = pd.DataFrame(index=path_routes.index)
    result_df['path_routes'] = path_routes
    
    # create "path_modes"
    df['path_modes'] = df['mode'].apply(lambda x: x.strip())
    result_df['path_modes'] = pd.DataFrame(df.groupby(group)['mode'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    # create "path_agencies"
    df['path_agencies'] = df['agency'].apply(lambda x: x.strip())
    result_df['path_agencies'] = pd.DataFrame(df.groupby(group)['agency'].apply(lambda x: "%s" % ' '.join(x).strip()))

    # Create "path_components"
    df['path_components'] = df['A_id']+" "+df['mode']+" "+df['route_id'] +"_"+ df['B_id']
    df['path_components'] = df['path_components'].apply(lambda x: x.strip())
    result_df['path_components'] = pd.DataFrame(df.groupby(group)['path_components'].apply(lambda x: "%s" % ' '.join(x).strip()))
    
    # stop components
    df['A_id'] = df['A_id'].astype('str')
    df['B_id'] = df['B_id'].astype('str')
    df['path_stops'] = df['A_id'] +" "+df['B_id']
    df['path_stops'] = df['path_stops'].apply(lambda x: x.strip())
    result_df['path_stops'] = pd.DataFrame(df.groupby(group)['path_stops'].apply(lambda x: "%s" % ' '.join(x).strip()))
    # drop repeated records (A_id and B_id overlap as origin and destination nodes for subsequent trips)
    result_df['path_stops'] = result_df['path_stops'].apply(lambda row: np.unique(row.split(' ')))
    # write out as space separated field
    result_df['path_stops'] = result_df['path_stops'].apply(lambda x: "%s" % ' '.join(x).strip())
    
    # Return ID field from index
    result_df['unique_id'] = result_df.index.get_level_values(0).values

    return result_df

# Load Data

- observed data (on-board survey [OBS])
- route and agency lookups

In [263]:
routes = pd.read_csv(r'../data/gtfs/agency_route_1.9.csv')

# Load observed and chosenpath_links; add new field designating 'model' or 'observed'
obs = load_df(data=obs_links_dir, unique_fields=['person_id','trip_list_id_num'], record_type='observed')

In [264]:
# Fill floats with integers where available, specifically for stop IDs
obs['A_id'] = obs['A_id'].fillna(0).astype('int')
obs['B_id'] = obs['B_id'].fillna(0).astype('int')

- chosenpaths links output from Fast-Trips

In [265]:
chosenpath_links = load_df(data=OUTPUT_DIR + r'\chosenpaths_links.csv', 
    unique_fields=['person_id','trip_list_id_num'], record_type='model', )

In [266]:
# For some reason there are a lot of duplicates
# let's drop them for now
chosenpath_links = chosenpath_links.drop_duplicates()

# Get the last iteration only
chosenpath_links = chosenpath_links[chosenpath_links['iteration'] == chosenpath_links['iteration'].max()]

In [267]:
# Add transit agency field to chosenpath_links and pathset_links, based on route_id
chosenpath_links = add_transit_agency(df=chosenpath_links, routes=routes)

In [268]:
chosenpath_links['A_id'] = chosenpath_links['A_id'].fillna(0).astype('int')
chosenpath_links['B_id'] = chosenpath_links['B_id'].fillna(0).astype('int')

In [269]:
observed_path = produce_path_fields(obs, group=['unique_id'])
modeled_path = produce_path_fields(chosenpath_links, group=['unique_id'])

# Make sure we only evaluate the overlapping unique_id records
observed_path = observed_path[observed_path['unique_id'].isin(modeled_path['unique_id'].values)]
modeled_path = modeled_path[modeled_path['unique_id'].isin(observed_path['unique_id'].values)]

# Ensure only single whitespaces separate comparison fields
for field in ['path_routes','path_modes','path_agencies', 'path_components', 'path_stops']:
    observed_path[field] = observed_path[field].apply(lambda row: ' '.join(row.split()))
    modeled_path[field] = modeled_path[field].apply(lambda row: ' '.join(row.split()))

In [270]:
pathset_paths = pd.read_csv(OUTPUT_DIR+ r'/pathset_paths.csv')
pathset_paths['unique_id'] = pathset_paths['person_id']

# Build comparison fields for pathset_links
- from the description column in pathset_paths

In [271]:
def mode_list(row):

    row_array = row.split(' ')
    
    mode_results = []
    
    for field in row_array:
        if field in transit_mode_list + ['transfer','walk_access','walk_egress']:
            mode_results.append(field)

    # convert from list into space-seperated string
    mode_results = ' '.join(mode_results).strip()

    return mode_results

In [272]:
def route_list(row):
    """
    if route_only=True, return only the route id from the trip id prepended with route info
    otherwise return full route_id_xyz, where xyz is the trip id
    """
    
    row_array = row.split(' ')
    
    route_results = []
    
    for i in xrange(len(row_array)):
        if row_array[i] in transit_mode_list:
            route_results.append("_".join(row_array[i+1].split('_')[:-1]))     # Drop last component of field (trip ID)
#                 route_results.append(row_array[i+1])
            
    route_results =  ' '.join(route_results).strip()
    
    return route_results

In [273]:
def stop_list(row):
    
    row_array = row.split(' ')
    
    # Get the list of transit
    trip_with_route_list = []
    stop_results = []
    
    for i in xrange(len(row_array)):
        if row_array[i] in transit_mode_list:
            # Save this as a field we don't want to include as a stop
            trip_with_route_list.append(row_array[i+1])
        if row_array[i] not in trip_with_route_list+transit_mode_list+['transfer','walk_access','walk_egress']:
            stop_results.append(row_array[i])
    
    stop_results = ' '.join(stop_results).strip()
    
    return stop_results

In [274]:
def agency_list(row):
    
    agency_results = []
    
    # Get list of agencies associated with each transit route
    row_array = row.split(' ')
    
    for route_id in row_array:
        agency_results.append(routes[routes['route_id'] == route_id]['agency_id'].values[0])
    
    agency_results = ' '.join(agency_results).strip()
    
    return agency_results

In [275]:
# Mode paths
pathset_paths['path_modes'] = pathset_paths['description'].apply(lambda row: mode_list(row))

In [276]:
# Route paths
pathset_paths['path_routes'] = pathset_paths['description'].apply(lambda row: route_list(row))

In [277]:
# Stops
pathset_paths['path_stops'] = pathset_paths['description'].apply(lambda row: stop_list(row))

In [278]:
# Agencies
pathset_paths['path_agencies'] = pathset_paths['path_routes'].apply(lambda row: agency_list(row))

In [336]:
# Create a stacked csv of observed trip links & model chosenpath_links; export for Tableau
chosenpath_links, obs = select_common_records(chosenpath_links, obs,'person_id')
chosenpaths_links_with_observed = append(chosenpath_links, obs)

### Add fields for Tableau visualization
- transfer from/to agency

In [337]:
# # add additional info about the transfer from and to route/agency for tableau maps
chosenpaths_links_with_observed['transfer_from_agency'] = chosenpaths_links_with_observed['agency'].shift(1)
chosenpaths_links_with_observed['transfer_to_agency'] = chosenpaths_links_with_observed['agency'].shift(-1)

chosenpaths_links_with_observed['transfer_from_route'] = chosenpaths_links_with_observed['route_id'].shift(1)
chosenpaths_links_with_observed['transfer_to_route'] = chosenpaths_links_with_observed['route_id'].shift(-1)

len(chosenpaths_links_with_observed)

86912

In [338]:
# Fill empty strings with 0 for origin and destination fields
chosenpaths_links_with_observed[['A_id','B_id']] = chosenpaths_links_with_observed[['A_id','B_id']].replace('',0)

In [339]:
chosenpaths_links_with_observed[['A_id','B_id']].head()

,A_id,B_id
0,1249,14663
1,14663,14673
2,14673,1356
3,1258,7414
4,7414,6886


- add lat/long values for stops and TAZ origins/destinations

In [340]:
# Add geography
# Get distance from TAZ to stop
walk_access_ft = pd.read_csv(r'R:\FastTrips\network_draft1.9\walk_access_ft.txt')[['taz','stop_id','dist',
                                                                                   'elevation_gain', 'indirectness']]

chosenpaths_links_with_observed[['A_id','B_id']] = chosenpaths_links_with_observed[['A_id','B_id']].astype('float')

In [341]:
# Join TAZ-centroid to stop ID distance, first for access trips (A_id as centroid, B_id as transit stop)
chosenpaths_links_with_observed = pd.merge(chosenpaths_links_with_observed, walk_access_ft, 
                                           left_on=['A_id','B_id'],right_on=['taz','stop_id'],how='left')


chosenpaths_links_with_observed['walk_access_dist'] = chosenpaths_links_with_observed['dist']
chosenpaths_links_with_observed['walk_access_elevation_gain'] = chosenpaths_links_with_observed['elevation_gain']
chosenpaths_links_with_observed['walk_access_indirectness'] = chosenpaths_links_with_observed['indirectness']

In [342]:
# Now join for distance from stop to TAZ centroid (egress)

# Drop existing fields from walk_access_ft for clarity
chosenpaths_links_with_observed.drop(['taz','stop_id','dist','elevation_gain','indirectness'],axis=1,inplace=True)
# for egress trips, A_id is stop ID and B_id is centroid, so reverse the merge fields for walk_access_ft
chosenpaths_links_with_observed = pd.merge(chosenpaths_links_with_observed, walk_access_ft, 
                                           left_on=['A_id','B_id'],right_on=['stop_id','taz'],how='left')
chosenpaths_links_with_observed['walk_egress_dist'] = chosenpaths_links_with_observed['dist']
chosenpaths_links_with_observed['walk_egress_elevation_gain'] = chosenpaths_links_with_observed['elevation_gain']
chosenpaths_links_with_observed['walk_egress_indirectness'] = chosenpaths_links_with_observed['indirectness']
chosenpaths_links_with_observed.drop(['taz','stop_id','dist'],axis=1,inplace=True)

In [343]:
len(chosenpaths_links_with_observed)

86912

In [344]:
# Add lat-long where it's missing for stops, using GTFS-based network data
# This will give lat long values associated with each A_id location
stops = pd.read_csv(r'R:\FastTrips\network_draft1.9\stops.txt')

# Join stop lat/lon for A_id stops
chosenpaths_links_with_observed = pd.merge(chosenpaths_links_with_observed,stops,left_on='A_id',right_on='stop_id',how='left')
chosenpaths_links_with_observed.rename(columns={'stop_lat':'A_lat', 'stop_lon':'A_lon'},inplace=True)

chosenpaths_links_with_observed = pd.merge(chosenpaths_links_with_observed,stops,left_on='B_id',right_on='stop_id',how='left')
chosenpaths_links_with_observed.rename(columns={'stop_lat':'B_lat','stop_lon':'B_lon'},inplace=True)

In [345]:
# Now all stops should have lat/long
# Next add lat and lon for TAZ fields

# Centroid locations for each TAZ
taz = pd.read_csv(r'R:\FastTrips\network_draft1.9\taz_coords.txt')

In [346]:
# For the origin TAZs
chosenpaths_links_with_observed = pd.merge(chosenpaths_links_with_observed,taz,left_on='A_id',right_on='taz',how='left')
chosenpaths_links_with_observed.rename(columns={'lat':'A_taz_lat','lon':'A_taz_lon'},inplace=True)

# For destination TAZs
chosenpaths_links_with_observed = pd.merge(chosenpaths_links_with_observed,taz,left_on='B_id',right_on='taz',how='left')
chosenpaths_links_with_observed.rename(columns={'lat':'B_taz_lat','lon':'B_taz_lon'},inplace=True)

In [347]:
# chosenpaths_links_with_observed[['A_id','B_id','A_lat','A_lon','B_lat','B_lon','A_taz_lat','A_taz_lon','B_taz_lat','B_taz_lon']]

In [348]:
# Fill the missing lat and lon fields with taz_lat and taz_lon
chosenpaths_links_with_observed['A_lat'].fillna(chosenpaths_links_with_observed['A_taz_lat'],inplace=True)
chosenpaths_links_with_observed['A_lon'].fillna(chosenpaths_links_with_observed['A_taz_lon'],inplace=True)

chosenpaths_links_with_observed['B_lat'].fillna(chosenpaths_links_with_observed['B_taz_lat'],inplace=True)
chosenpaths_links_with_observed['B_lon'].fillna(chosenpaths_links_with_observed['B_taz_lon'],inplace=True)

In [349]:
# Some access and egress distances are missing because 
# walk_access_ft doesn't include all zone to stop pairs
# Calculate straight-line distance for all of these missing pairs
# df = chosenpaths_links_with_observed[(chosenpaths_links_with_observed['walk_access_dist'].isnull()) &\
#                                      (chosenpaths_links_with_observed['mode'] == 'walk_access')]
# df[['A_id','B_id','A_lat','A_lon','B_lat','B_lon','distance','record_type']].iloc[0]

# haversine(-122.41848,37.79069,-122.39518,37.77638)

In [350]:
chosenpaths_links_with_observed[['A_id','B_id','A_lat','A_lon','B_lat','B_lon']].head()

,A_id,B_id,A_lat,A_lon,B_lat,B_lon
0,1249.00000,14663.00000,37.58460,-122.34119,37.58103,-122.34706
1,14663.00000,14673.00000,37.58103,-122.34706,37.44364,-122.16520
2,14673.00000,1356.00000,37.44364,-122.16520,37.44673,-122.15701
3,1258.00000,7414.00000,37.57336,-122.33748,37.56974,-122.33609
4,7414.00000,6886.00000,37.56974,-122.33609,37.44334,-122.16432


In [351]:
chosenpaths_links_with_observed.to_csv(OUTPUT_DIR + '/' + 'chosenpaths_links_with_observed.csv',index=False)

# Compare Modeled v Observed
### Produce new output file path_comparison.csv

In [231]:
# Combine the observed and modeled path files
df = pd.merge(observed_path, modeled_path, on='unique_id',suffixes=("_observed","_model"))

- compare routes, modes, and agencies

In [232]:
# Build list of routes used in observed and modeled trips
df['model_path_route_list'] = df['path_routes_model'].apply(lambda x: x.split(" "))
df['obs_path_route_list'] = df['path_routes_observed'].apply(lambda x: x.split(" "))

# Build list of modes used in observed and modeled trips
df['model_path_mode_list'] = df['path_modes_model'].apply(lambda x: x.split(" "))
df['obs_path_mode_list'] = df['path_modes_observed'].apply(lambda x: x.split(" "))

# Build list of transit agencies used in observed and modeled trips
df['model_path_agencies_list'] = df['path_agencies_model'].apply(lambda x: x.split(" "))
df['obs_path_agencies_list'] = df['path_agencies_observed'].apply(lambda x: x.split(" "))

# Build list of stops used in observed and modeled trips
df['model_path_stops_list'] = df['path_stops_model'].apply(lambda x: x.split(" "))
df['obs_path_stops_list'] = df['path_stops_observed'].apply(lambda x: x.split(" "))


In [233]:
obs[obs['unique_id'] == '10---Caltrain---2014']

,person_id,trip_list_id_num,linkmode,A_id,B_id,linknum,mode,route_id,agency,record_type,unique_id,path_routes,path_modes,path_agencies,path_components,path_stops
3,10---Caltrain---2014,2,access,1249,14661,0,KNR_access,,,observed,10---Caltrain---2014,,KNR_access,,1249 KNR_access _14661,1249 14661
4,10---Caltrain---2014,2,transit,14661,14673,1,commuter_rail,caltrain_unknown,caltrain,observed,10---Caltrain---2014,caltrain_unknown,commuter_rail,caltrain,14661 commuter_rail caltrain_unknown_14673,14661 14673
5,10---Caltrain---2014,2,egress,14673,1356,2,walk_egress,,,observed,10---Caltrain---2014,,walk_egress,,14673 walk_egress _1356,14673 1356


In [234]:
chosenpath_links[chosenpath_links['unique_id'] == '10---Caltrain---2014'].columns

Index([u'index', u'person_id', u'person_trip_id', u'pathnum', u'linkmode', u'A_seq', u'B_seq', u'linknum', u'A_id', u'B_id', u'trip_id', u'route_id', u'mode', u'chosen', u'new_A_time', u'new_B_time', u'new_linktime min', u'new_waittime min', u'missed_xfer', u'distance', u'sim_cost', u'board_time', u'overcap', u'alight_time', u'iteration', u'record_type', u'unique_id', u'agency_id', u'agency', u'path_routes', u'path_modes', u'path_agencies', u'path_components', u'path_stops'], dtype='object')

In [235]:
chosenpath_links[chosenpath_links['unique_id'] == '10---Caltrain---2014'].groupby('record_type').count()

,index,person_id,person_trip_id,pathnum,linkmode,A_seq,B_seq,linknum,A_id,B_id,...,alight_time,iteration,unique_id,agency_id,agency,path_routes,path_modes,path_agencies,path_components,path_stops
record_type,,,,,,,,,,,,,,,,,,,,,
model,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [236]:
 # Isolate transit modes only, because all trips should have walk & transfer components
    
df['model_transit_modes'] = df['model_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])
df['obs_transit_modes'] = df['obs_path_mode_list'].apply(
    lambda row: [element for element in row if element not in non_transit_modes])

In [237]:
df[['obs_transit_modes','model_transit_modes']].head()

,obs_transit_modes,model_transit_modes
0,[commuter_rail],[commuter_rail]
1,[commuter_rail],"[local_bus, local_bus]"
2,[heavy_rail],[heavy_rail]
3,"[local_bus, commuter_rail]",[commuter_rail]
4,[heavy_rail],[local_bus]


In [238]:
# Find the intersection between the chosen model/observed paths using different criteria
# Which are in common between model and observed?

# transit route IDs only
df.apply(lambda row: all(i in row['model_path_route_list'] for i in row['obs_path_route_list']), axis=1)
df['routes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_route_list'], df['obs_path_route_list'])]

# stops only
df.apply(lambda row: all(i in row['model_path_stops_list'] for i in row['obs_path_stops_list']), axis=1)
df['stops_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_stops_list'], df['obs_path_stops_list'])]

# All Modes (including transfer, access/egress)
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['all_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_mode_list'], df['obs_path_mode_list'])]

# Transit modes only (type of vehicle taken and number of boardings)
df.apply(lambda row: all(i in row['model_path_mode_list'] for i in row['obs_path_mode_list']), axis=1)
df['transit_modes_intersection'] = [list(set(a).intersection(set(b))) for a, b in zip(df['model_transit_modes'], df['obs_transit_modes'])]

# Agency Intersection
df.apply(lambda row: all(i in row['model_path_agencies_list'] for i in row['obs_path_agencies_list']), axis=1)
df['agency_intersection'] = \
    [list(set(a).intersection(set(b))) for a, b in zip(df['model_path_agencies_list'], 
        df['obs_path_agencies_list'])]

In [239]:
df[df.unique_id == '1714---Tri-Delta---2014'][['path_routes_observed','path_routes_model']]

,path_routes_observed,path_routes_model
1859,bart_unknown tri_delta_transit_390TRI_,bart_BART_YEL tri_delta_transit_387TRI_


In [240]:
complete_agency_match[complete_agency_match.unique_id == '1714---Tri-Delta---2014']['complete_agency_match']

1859    1
Name: complete_agency_match, dtype: int64

In [241]:
# Exact Match of path routes, modes, & components
# Isolate rows (trip legs) with matching path routes
complete_route_match = df[df['path_routes_observed'] == df['path_routes_model']]
complete_mode_match = df[df['path_modes_observed'] == df['path_modes_model']]
complete_agency_match = df[df['path_agencies_observed'] == df['path_agencies_model']]
complete_stop_match = df[df['path_stops_observed'] == df['path_stops_model']]

complete_route_match['complete_route_match'] = 1
complete_mode_match['complete_mode_match'] = 1
complete_agency_match['complete_agency_match'] = 1
complete_stop_match['complete_stop_match'] = 1

# Add new columns to the larger dataframe indicating if the row is a complete match
df = pd.merge(df, complete_mode_match[['unique_id','complete_mode_match']], how='left', on='unique_id')


df = pd.merge(df, complete_route_match[['unique_id','complete_route_match']], how='left', on='unique_id')
df = pd.merge(df, complete_agency_match[['unique_id','complete_agency_match']], how='left', on='unique_id')
df = pd.merge(df, complete_stop_match[['unique_id','complete_stop_match']], how='left', on='unique_id')

for field in ['mode','route','agency', 'stop']:
    df['complete_'+field+'_match']=  df['complete_'+field+'_match'].replace('nan',0)

In [242]:
# Now we find the percent of trips with matching routes or partial matching routes

# Join the filtered data to the original results
df['common_route_count'] = [len(row) for row in df['routes_intersection']]
df['common_mode_count'] = [len(row) for row in df['all_modes_intersection']]
df['common_transit_mode_count'] = [len(row) for row in df['transit_modes_intersection']]
df['common_agency_count'] = [len(row) for row in df['agency_intersection']]
df['common_stop_count'] = [len(row) for row in df['stops_intersection']]

# How many rows have at least one mode in common?
df['partial_mode_match'] = [1 if row > 0 else 0 for row in df['common_mode_count']]
df['partial_transit_mode_match'] = [1 if row > 0 else 0 for row in df['common_transit_mode_count']]
df['partial_route_match'] = [1 if row > 0 else 0 for row in df['common_route_count']]
df['partial_agency_match'] = [1 if row > 0 else 0 for row in df['common_agency_count']]
df['partial_stop_match'] = [1 if row > 0 else 0 for row in df['common_stop_count']]

# Export
df.to_csv(OUTPUT_DIR + r'/path_intersection.csv')

In [243]:
df[df.unique_id == '1714---Tri-Delta---2014']['agency_intersection']

1859    [tri_delta_transit, bart]
Name: agency_intersection, dtype: object

In [244]:
df[df.unique_id == '1714---Tri-Delta---2014']['partial_agency_match']

1859    1
Name: partial_agency_match, dtype: int64

- check if observed path is in pathset

In [245]:
# pathset_links
pathset_paths.columns

Index([u'person_id', u'person_trip_id', u'pathdir', u'pathmode', u'pathnum', u'pf_cost', u'pf_probability', u'description', u'chosen', u'missed_xfer', u'sim_cost', u'ln_PS', u'logsum_component', u'logsum', u'probability', u'iteration', u'simulation_iteration', u'unique_id', u'path_modes', u'path_routes', u'path_stops', u'path_agencies'], dtype='object')

In [246]:
newdf = pd.merge(observed_path, pathset_paths, how='left',
          left_on=['unique_id',comparison_field],right_on=['unique_id',comparison_field], suffixes=['_obs','_pathset'])
newdf[newdf.unique_id == '1714---Tri-Delta---2014']

,path_routes_obs,path_modes_obs,path_agencies,path_components,path_stops_obs,unique_id,person_id,person_trip_id,pathdir,pathmode,...,sim_cost,ln_PS,logsum_component,logsum,probability,iteration,simulation_iteration,path_modes_pathset,path_routes_pathset,path_stops_pathset
3447,bart_unknown tri_delta_transit_390TRI_,walk_access heavy_rail transfer local_bus walk...,bart tri_delta_transit,623 walk_access _102616 102616 heavy_rail bart...,0 102616 102639 2199 623,1714---Tri-Delta---2014,1714---Tri-Delta---2014,56383.00000,2.00000,walk-transit-walk,...,125.00305,0.00000,0.00000,0.00000,1.00000,1.00000,2.00000,walk_access heavy_rail transfer local_bus walk...,bart_BART_YEL tri_delta_transit_387TRI_,623 16514 16536 102616 102810 2199
3448,bart_unknown tri_delta_transit_390TRI_,walk_access heavy_rail transfer local_bus walk...,bart tri_delta_transit,623 walk_access _102616 102616 heavy_rail bart...,0 102616 102639 2199 623,1714---Tri-Delta---2014,1714---Tri-Delta---2014,56383.00000,2.00000,walk-transit-walk,...,151.28305,0.00000,0.00000,0.00000,0.00000,1.00000,2.00000,walk_access heavy_rail transfer local_bus walk...,bart_BART_YEL tri_delta_transit_387TRI_,623 16514 16536 102616 102810 2199


In [247]:
pathset_paths[pathset_paths.unique_id == '1714---Tri-Delta---2014'][['unique_id','path_agencies']]

,unique_id,path_agencies
12649,1714---Tri-Delta---2014,bart tri_delta_transit
12650,1714---Tri-Delta---2014,bart tri_delta_transit


In [248]:
observed_path[observed_path.unique_id == '1714---Tri-Delta---2014'][['unique_id','path_agencies']].path_agencies.values[0]

'bart tri_delta_transit'

In [249]:
chosenpath_links[chosenpath_links.unique_id == '1714---Tri-Delta---2014']

,index,person_id,person_trip_id,pathnum,linkmode,A_seq,B_seq,linknum,A_id,B_id,...,iteration,record_type,unique_id,agency_id,agency,path_routes,path_modes,path_agencies,path_components,path_stops
9609,9609,1714---Tri-Delta---2014,56383,0,access,-1,-1,0,623,16514,...,1,model,1714---Tri-Delta---2014,,,,walk_access,,623 walk_access _16514,623 16514
9610,9610,1714---Tri-Delta---2014,56383,0,transit,10,25,1,16514,16536,...,1,model,1714---Tri-Delta---2014,bart,bart,bart_BART_YEL,heavy_rail,bart,16514 heavy_rail bart_BART_YEL_16536,16514 16536
9611,9611,1714---Tri-Delta---2014,56383,0,transfer,-1,-1,2,16536,102616,...,1,model,1714---Tri-Delta---2014,,,,transfer,,16536 transfer _102616,16536 102616
9612,9612,1714---Tri-Delta---2014,56383,0,transit,1,20,3,102616,102810,...,1,model,1714---Tri-Delta---2014,tri_delta_transit,tri_delta_transit,tri_delta_transit_387TRI_,local_bus,tri_delta_transit,102616 local_bus tri_delta_transit_387TRI__102810,102616 102810
9613,9613,1714---Tri-Delta---2014,56383,0,egress,-1,-1,4,102810,2199,...,1,model,1714---Tri-Delta---2014,,,,walk_egress,,102810 walk_egress _2199,102810 2199


In [250]:
pathset_paths[pathset_paths.unique_id == '1714---Tri-Delta---2014'][['unique_id','path_agencies']].iloc[0].path_agencies

'bart tri_delta_transit'

In [251]:
## Add a field to the new_pathset that lists the pathnum
# pathset_links['pathnum'] = pathset_links.index.get_level_values(1)

# Join paths based on comparison_field, as defined in script header
# Resulting df is merge of all observed paths that have a corresponding path in the pathset for their unique_id
# observed paths with no corresponding path in pathset will have NaN for fields "_pathset" suffix
newdf = pd.merge(observed_path, pathset_paths, how='left',
          left_on=['unique_id',comparison_field],right_on=['unique_id',comparison_field], suffixes=['_obs','_pathset'])

# Join this data with the pathset path file to get pf_probability associated with the observed path
# newdf = pd.merge(df, pathset_paths[['unique_id','pathnum','pf_probability']], 
#                  left_on=['unique_id','pathnum'], right_on=['unique_id','pathnum'],
#                  how='left')

# Fields with NaN marked as no match since no matching path was found in pathset
newdf['probability'] = newdf['pf_probability'].fillna('no_match')

# Grab the highest and lowest probabilities from pathset paths
# want to test that the observed path has a reasonably high probability
max_prob = newdf.groupby('unique_id').max()['probability']
min_prob = newdf.groupby('unique_id').min()['probability']

# Reshape those results and export to dataframe
prob_export = pd.DataFrame([max_prob,min_prob]).T
prob_export.columns = ['max_prob','min_prob']


# Reformat at binary to indicate whether a path was found in the pathset
prob_export['path_exists'] = prob_export['max_prob'].apply(lambda row_value: 0 if row_value == 'no_match' else 1)
prob_export.to_csv('temp_prob_export.csv')

# Create a variabale to indicate if the max probability of the observed path
# is over a given threshold, as defined at top of script
try:
    prob_export.ix[prob_export['max_prob'] >= threshold, 'above_threshold'] = 1
    prob_export.ix[prob_export['max_prob'] < threshold, 'above_threshold'] = 0
    prob_export.ix[prob_export['max_prob'] == 'no_match', 'above_threshold'] = -1
except:
    pass

# export the results probabilities, path_existence, threshold data,
# also add the modeled and observed (chosen) path characteristics
prob_export['unique_id'] = prob_export.index

tempdf = pd.merge(observed_path,modeled_path,on='unique_id',suffixes=['_obs','_model'],how='left')
export_df = pd.merge(prob_export,tempdf,on='unique_id',how='left')

export_df['person_id'] = export_df['unique_id'].apply(lambda row: row.split("_")[0])
export_df['trip_list_id_num'] = export_df['unique_id'].apply(lambda row: row.split("_")[-1])

export_df.to_csv(OUTPUT_DIR + '\path_comparison.csv', index=False)

In [252]:
export_df[export_df.unique_id == '1714---Tri-Delta---2014']['path_exists']

1859    1
Name: path_exists, dtype: int64

In [253]:
export_df['path_exists'].mean()

0.3817312993934938

In [254]:
export_df[export_df['max_prob'] != 'no_match'].mean()

path_exists       1.00000
above_threshold   0.52985
dtype: float64

In [255]:
df['complete_route_match'].mean()

0.009832751332475648

In [256]:
df['complete_stop_match'].mean()

0.2018011394964161

In [257]:
df['complete_agency_match'].mean()

0.3072964528579305

# Add time fields
- OBS data has no time fields, but we still want to query by time of day so copy FT